# Практическое задание к уроку "Рекуррентные нейронные сети"

Попробуйте обучить нейронную сеть RNN/LSTM/GRU на любом другом датасете (любимый временной ряд, текст на русском (другом языке) как генератор или классификатор, или прилагаемый набор airline-passengers - пасажиропоток для авиалиний).


In [1]:
import numpy as np
from keras.layers import Dense, Activation
from keras.layers.recurrent import SimpleRNN, LSTM, GRU
from keras.models import Sequential

# построчное чтение из примера с текстом 
with open("witcher.txt", 'rb') as f:
    lines = []
    for line in f:
        line = line.strip().lower().decode("utf8", "ignore")
        if len(line) == 0:
            continue
        lines.append(line)

text = " ".join(lines)
chars = set([c for c in text])
n_chars = len(chars)

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [2]:
text[0:1000]

'Глава 1 Чтобы зарабатывать на жизнь в качестве настоящего гонца, - любил говаривать Аплегатт поступающим на службу юнцам, - требуются, во-первых, золотая голова и, во-вторых, железная задница. Золотая голова, - поучал Аплегатт молодых гонцов, - необходима, поскольку под одеждой, в привязанной к голой груди плоской кожаной суме гонец возит только малозначительные сообщения, которые не опасаясь можно доверить ненадежной бумаге либо пергаменту. По-настоящему же важные, секретные известия, от которых многое зависит, гонец должен запомнить и повторить кому следует. Слово в слово. А это порой бывают непростые слова. Их и выговорить-то трудно, не то что запомнить. А чтобы запомнить и, повторяя, не ошибиться, надобна воистину золотая голова. Что же касается железной задницы, так это любой гонец очень даже скоро почувствует сам, стоит ему провести в седле три дня и три ночи, протрястись сто, а то и двести верст по большакам, а ежели понадобится, то и по бездорожью. Ну, само собой, сидишь в сед

In [3]:
chars

{'\t',
 ' ',
 '!',
 '"',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '<',
 '>',
 '?',
 '^',
 '`',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 'А',
 'Б',
 'В',
 'Г',
 'Д',
 'Е',
 'Ж',
 'З',
 'И',
 'Й',
 'К',
 'Л',
 'М',
 'Н',
 'О',
 'П',
 'Р',
 'С',
 'Т',
 'У',
 'Ф',
 'Х',
 'Ц',
 'Ч',
 'Ш',
 'Щ',
 'Ы',
 'Ь',
 'Э',
 'Ю',
 'Я',
 'а',
 'б',
 'в',
 'г',
 'д',
 'е',
 'ж',
 'з',
 'и',
 'й',
 'к',
 'л',
 'м',
 'н',
 'о',
 'п',
 'р',
 'с',
 'т',
 'у',
 'ф',
 'х',
 'ц',
 'ч',
 'ш',
 'щ',
 'ъ',
 'ы',
 'ь',
 'э',
 'ю',
 'я',
 'ё'}

In [4]:
len(chars)

120

In [5]:
# создание индекса символов и reverse mapping
char2index = {c: i for i, c in enumerate(chars)}
index2char = {i: c for i, c in enumerate(chars)}

In [6]:
# для удобства выберете фиксированную длину последовательность 10 символов 
maxlen, step = 10, 1
input_chars, label_chars = [], []

# конвертация data в серии разных maxlen-length последовательностей
for i in range(0, len(text) - maxlen, step):
    input_chars.append(text[i: i + maxlen])
    label_chars.append(text[i + maxlen])

In [7]:
input_chars[:10]

['Глава 1 Чт',
 'лава 1 Что',
 'ава 1 Чтоб',
 'ва 1 Чтобы',
 'а 1 Чтобы ',
 ' 1 Чтобы з',
 '1 Чтобы за',
 ' Чтобы зар',
 'Чтобы зара',
 'тобы зараб']

In [8]:
label_chars[:10]

['о', 'б', 'ы', ' ', 'з', 'а', 'р', 'а', 'б', 'а']

In [9]:
# Вычисление one-hot encoding входных последовательностей X и следующего символа (the label) y

X = np.zeros((len(input_chars), maxlen, n_chars))
y = np.zeros((len(input_chars), n_chars))

for i, input_char in enumerate(input_chars):
    for j, ch in enumerate(input_char):
        X[i, j, char2index[ch]] = 1
    y[i, char2index[label_chars[i]]] = 1

In [10]:
char2index['p'], char2index['r']

(17, 2)

In [11]:
X[0, :2]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.,

In [12]:
# установка ряда метапареметров для нейронной сети и процесса тренировки
batch_size = 128
hidden_size = 64 #128
num_iterations = 25 # 25 должно быть достаточно
num_epochs = 1
num_preds = 100

In [13]:
model = Sequential()
model.add(
    GRU(
        hidden_size,
        return_sequences=False,
        input_shape=(maxlen, n_chars),
        unroll=True
    )
)
model.add(Dense(n_chars, activation='softmax'))
model.compile(loss="categorical_crossentropy",
              optimizer="rmsprop")

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru (GRU)                    (None, 64)                35520     
_________________________________________________________________
dense (Dense)                (None, 120)               7800      
Total params: 43,320
Trainable params: 43,320
Non-trainable params: 0
_________________________________________________________________


In [15]:
# выполнение серий тренировочных и демонстрационных итераций 
for iteration in range(num_iterations):
    # для каждой итерации запуск передачи данных в модель 
    print()
    print("=" * 50)
    print(f'Итерация #: {iteration}')
    model.fit(X, y,
              batch_size=batch_size,
              epochs=num_epochs)

    # Select a random example input sequence
    test_idx = np.random.randint(len(input_chars))
    test_chars = input_chars[test_idx]

    # для числа шагов предсказаний использование текущей тренируемой модели 
    # конструирование one-hot encoding для тестирования input и добавление предсказания.
    print(f'Генерация из {test_chars}')
    print(test_chars, end="")
    for i in range(num_preds):

        # здесь one-hot encoding.
        X_test = np.zeros((1, maxlen, n_chars))
        for j, ch in enumerate(test_chars):
            X_test[0, j, char2index[ch]] = 1

        # осуществление предсказания с помощью текущей модели.
        pred = model.predict(X_test, verbose=0)[0]
        y_pred = index2char[np.argmax(pred)]

        # вывод предсказания добавленного к тестовому примеру 
        print(y_pred, end="")

        # инкрементация тестового примера содержащего предсказание
        test_chars = test_chars[1:] + y_pred


Итерация #: 0
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: invalid syntax (tmpt1zu_q9y.py, line 81)
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: invalid syntax (tmpt1zu_q9y.py, line 81)
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: invalid syntax (tmp0ujk0mjs.py, line 48)
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosi

MemoryError: Unable to allocate 11.8 GiB for an array with shape (2635521, 10, 120) and data type float32

Опишите, какой результат вы получили? Что помогло вам улучшить ее точность?

В итоге сеть GRU, построенная по тексту книг "Ведьмак" не выдала осмысленных результатов. Наилучшая точность была достигнута при использовании 64 нейронов скрытого слоя. Добавление нового слоя сопровождалось ошибкой. Количество элементов матрицы не влезло в оперативную память.

Итерация #: 0  

Генерация из ихся оборо
ихся оборону с нем поднет и подомнать в конет поднет и подомнать в конет поднет и подомнать в конет поднет и п

Итерация #: 1
  
Генерация из ак навозна
ак навозная и подомнила она не подомнил он не подомнил он не подомнил он не подомнил он не подомнил он не подо

Итерация #: 2

Генерация из наша. - Ва
наша. - Вас в только с только с только с только с только с только с только с только с только с только с только

Итерация #: 3

Генерация из аленькими 
аленькими в столо в столо в столо в столо в столо в столо в столо в столо в столо в столо в столо в столо в ст

Итерация #: 4

Генерация из ь гонцы уж
ь гонцы уже не подобность подобность подобность подобность подобность подобность подобность подобность подобно

Итерация #: 5

Генерация из ни на что.
ни на что. - Не под на волосы в столь под на волосы в столь под на волосы в столь под на волосы в столь под на

Итерация #: 6

Генерация из ащили. А е
ащили. А если не поднял он не поднял он не поднял он не поднял он не поднял он не поднял он не поднял он не по

Итерация #: 7

Генерация из ми огневым
ми огневым поднял он не подовина и поднял он не подовина и поднял он не подовина и поднял он не подовина и под

Итерация #: 8

Генерация из группа. Ви
группа. Видел следует под столовать стану стану стану стану стану стану стану стану стану стану стану стану ст

Итерация #: 9

Генерация из , двумя, м
, двумя, может столько не под столовой в столовой в столовой в столовой в столовой в столовой в столовой в сто